In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForCausalLM
import torch
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

In [3]:
herbert_model = AutoModelForMaskedLM.from_pretrained("allegro/herbert-large-cased")
herbert_tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")

In [4]:
xlm_model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")
xlm_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [5]:
mbert_model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")
mbert_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [6]:
def predict_case(model, tokenizer, sentence, k):
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    mask_token_logits = logits[0, mask_token_index, :]

    top_tokens = torch.topk(mask_token_logits, k, dim=1).indices[0].tolist()
    predictions = [tokenizer.decode([token]).strip() for token in top_tokens]
    
    return predictions

In [7]:
def prepare_sentence(sentence, model_name):
    if model_name in ['herbert', 'xlm']:
        return sentence.replace("{mask}", "<mask>")
    else:
        return sentence.replace("{mask}", "[MASK]")

In [8]:
def print_predictions(base_sentences, num_predictions):
    for sentence in base_sentences:
        # Przygotowanie zdań dla każdego modelu
        sentences_herbert = prepare_sentence(sentence, 'herbert')
        sentences_xlm = prepare_sentence(sentence, 'xlm')
        sentences_mbert = prepare_sentence(sentence, 'mbert')
        
        print(f"\nSentence: {sentence}")
        print("\nHerBERT: ", predict_case(herbert_model, herbert_tokenizer, sentences_herbert, num_predictions))
        print("XLM: ", predict_case(xlm_model, xlm_tokenizer, sentences_xlm, num_predictions))
        print("Multilingual BERT: ", predict_case(mbert_model, mbert_tokenizer, sentences_mbert, num_predictions))
        print()

### Testing polish cases

In [10]:
sentences = [
    "Warszawa to największe {mask}.", # mianownik
    "Musiałem oddać samochód do mechanika więc teraz funkcjonuję bez {mask}.", # dopełniacz
    "Wczoraj w zoo przyglądałem się ogromnemu {mask}.", # celownik
    "Wieczorem gdy robi się ciemno, zawsze do nauki zapalam {mask}.", # biernik
    "Na zajęcia z przetwarzania języka naturalnego zawsze przychodzę z {mask}.", #narzędnik
    "Polska posiada orła w swoim {mask}.", # miejscownik
    "Zjadłeś całą kiełbasę! Ty denerwujący {mask}!" # wołacz
]

In [11]:
print_predictions(sentences, 5)


Sentence: Warszawa to największe {mask}.

HerBERT:  ['miasto', 'miasta', '.', 'lotnisko', 'miast']
XLM:  ['miasto', 'miasta', '', 'Miasto', 'city']
Multilingual BERT:  ['miasto', 'miasta', 'woj', 'Warszawa', 'to']


Sentence: Musiałem oddać samochód do mechanika więc teraz funkcjonuję bez {mask}.

HerBERT:  ['problemu', 'samochodu', 'niego', 'auta', 'problemów']
XLM:  ['problemu', 'problemów', 'zmian', 'niego', 'ruchu']
Multilingual BERT:  ['tego', 'to', 'niego', 'tym', 'co']


Sentence: Wczoraj w zoo przyglądałem się ogromnemu {mask}.

HerBERT:  ['.', 'zamieszaniu', 'dziecku', 'stworzeniu', 'zoo']
XLM:  ['zoo', 'parku', 'muzeum', 'polu', 'domu']
Multilingual BERT:  ['stylu', 'życiu', 'zoo', 'sensu', 'św']


Sentence: Wieczorem gdy robi się ciemno, zawsze do nauki zapalam {mask}.

HerBERT:  ['światło', 'papierosa', 'świece', 'światła', 'ogień']
XLM:  ['światło', 'telefon', 'komputer', 'alarm', 'akumulator']
Multilingual BERT:  ['się', 'go', '!', 'słowa', 'siebie']


Sentence: Na zajęc

### Testing long-range relationships

In [13]:
sentences = [
    "Szarlotka mojej babci wyszła znakomicie, chociaż zbytnio mnie to nie dziwi, gdyż od zawsze {mask} wspaniałe ciasta.",
    "Gdy uczestnik wszedł do pomieszczenia dla pracowników, ochroniarz momentalnie podbiegł do {mask}.",
    "Skoro wygrała konkurs piękności, z pewnością musi być bardzo {mask}."
]

In [14]:
print_predictions(sentences, 5)


Sentence: Szarlotka mojej babci wyszła znakomicie, chociaż zbytnio mnie to nie dziwi, gdyż od zawsze {mask} wspaniałe ciasta.

HerBERT:  ['robiła', 'robi', 'lubiła', 'piekła', 'robię']
XLM:  ['miałam', 'były', 'robi', 'lubię', 'miała']
Multilingual BERT:  ['to', 'ma', 'jest', 'miała', '-']


Sentence: Gdy uczestnik wszedł do pomieszczenia dla pracowników, ochroniarz momentalnie podbiegł do {mask}.

HerBERT:  ['niego', 'drzwi', 'wejścia', 'okna', 'poszkodowanego']
XLM:  ['niego', 'środka', 'drzwi', 'budynku', 'nich']
Multilingual BERT:  ['tzw', 'niego', 'góry', 'budynku', 'samolotu']


Sentence: Skoro wygrała konkurs piękności, z pewnością musi być bardzo {mask}.

HerBERT:  ['szczęśliwa', 'zadowolona', 'popularna', 'atrakcyjna', 'piękna']
XLM:  ['zadowolona', 'piękna', 'zadowolony', 'stara', 'ważna']
Multilingual BERT:  ['dobrze', 'dużą', 'wiele', 'znana', 'średniej']



### Testing real-world knowledge

In [17]:
sentences = [
    "Ściana komórkowa zbudowana jest z mureiny, więc musi to być komórka {mask}.",
    "Wybieram się do {mask}, najludniejszego państwa świata.",
    "Ziemia jest {mask} w kolejności planetą od słońca."
]

In [18]:
print_predictions(sentences, 5)


Sentence: Ściana komórkowa zbudowana jest z mureiny, więc musi to być komórka {mask}.

HerBERT:  ['ludzka', 'DNA', 'żywa', 'czysta', 'właściwa']
XLM:  ['.', 'ciepła', 'domu', ',', '...']
Multilingual BERT:  ['tzw', 'ta', 'np', 'w', 'typu']


Sentence: Wybieram się do {mask}, najludniejszego państwa świata.

HerBERT:  ['Indii', 'Chin', 'Japonii', 'Australii', 'Meksyku']
XLM:  ['Iraku', 'USA', 'Tokio', 'Peru', 'Zimbabwe']
Multilingual BERT:  ['świata', 'tego', 'Europy', 'swojego', 'nowego']


Sentence: Ziemia jest {mask} w kolejności planetą od słońca.

HerBERT:  ['drugą', 'trzecią', 'czwartą', 'piątą', 'szóstą']
XLM:  ['drugą', 'pierwszą', 'większą', 'druga', 'dalej']
Multilingual BERT:  ['drugą', 'druga', 'pierwszą', 'się', 'drugim']



### Testing zero-shot learning

In [20]:
sentences = [
    "'Wczoraj na urodzinach mojej babci bawiłem się jak nigdy dotąd! Bardzo szybko zleciał mi czas, zdecydowanie nie żałuję, że pojechałem' Wypowiedź ta jest zdecydowanie {mask}.",
    "'To był najgorszy koncert, na którym byłem, muzycy spóźnili się, a dźwięk był fatalny, co całkowicie zepsuło atmosferę.' Wypowiedź ta jest zdecydowanie {mask}.",
    "'Jedzenie w tej restauracji było pyszne! Każda potrawa była starannie przygotowana i podana z niezwykłą estetyką, co sprawiło, że chciałem wracać tam z przyjemnością.' Wypowiedź ta jest zdecydowanie {mask}.",
    "'Nie polecam tej książki, straciłem czas, ponieważ fabuła była chaotyczna, a postacie płaskie i nieciekawe, przez co nie mogłem się skupić na czytaniu.' Wypowiedź ta jest zdecydowanie {mask}.",
    "'Film był dość nudny, nie polecam, ponieważ tempo akcji było wolne, a dialogi mało interesujące, przez co odczuwałem znużenie.' Wypowiedź ta jest zdecydowanie {mask}."
]

In [21]:
print_predictions(sentences, 5)


Sentence: 'Wczoraj na urodzinach mojej babci bawiłem się jak nigdy dotąd! Bardzo szybko zleciał mi czas, zdecydowanie nie żałuję, że pojechałem' Wypowiedź ta jest zdecydowanie {mask}.

HerBERT:  ['prawdziwa', 'pozytywna', 'oryginalna', 'najlepsza', 'wyjątkowa']
XLM:  ['dobra', 'najlepsza', 'piękna', 'ważna', 'moja']
Multilingual BERT:  ['słowa', 'znana', 'sama', 'folk', '##łowa']


Sentence: 'To był najgorszy koncert, na którym byłem, muzycy spóźnili się, a dźwięk był fatalny, co całkowicie zepsuło atmosferę.' Wypowiedź ta jest zdecydowanie {mask}.

HerBERT:  ['prawdziwa', 'pozytywna', 'najlepsza', 'oficjalna', 'oryginalna']
XLM:  ['dobra', 'najlepsza', 'ważna', 'kontra', 'moja']
Multilingual BERT:  ['słowa', 'znana', '##łowa', 'stała', 'ta']


Sentence: 'Jedzenie w tej restauracji było pyszne! Każda potrawa była starannie przygotowana i podana z niezwykłą estetyką, co sprawiło, że chciałem wracać tam z przyjemnością.' Wypowiedź ta jest zdecydowanie {mask}.

HerBERT:  ['prawdziwa', 'p

## Questions

### 1) Which of the models produced the best results?

Among the tested models, herBERT performed the best. This was an expected outcome, as herBERT was trained exclusively on Polish data, which allows it to better understand the specific characteristics of the Polish language.

### 2) Was any of the models able to capture Polish grammar? 

The herBERT model can accurately match words in all Polish cases. The XLM-RoBERTa-based model also achieved good results, though it was not as consistent in every case as herBERT.

### 3) Was any of the models able to capture long-distant relationships between the words?

Both herBERT and XLM-Roberta performed strongly in this task, demonstrating high accuracy in correct matching. Despite the relatively short sentence lengths, multilingual BERT struggled to respond appropriately.

### 4) Was any of the models able to capture world knowledge?

The models' knowledge of the world is strongly limited, but they are sometimes able to make accurate matches. For example, herBERT correctly identified India as the most populous country in the world, with China in second place. This is interesting, as China population was recently exceeded by India – this could indicate that the model was trained on relatively recent data, although it could also be a coincidence.
It is also worth noting that all models gave the same incorrect answer to a question about the order of planets

### 5) Was any of the models good at doing zero-shot classification?

In this task, the models struggled to accurately evaluate the emotional tone of statements. They often asserted that a given opinion was "prawdziwa" or "słuszna", even when it was clearly negatively charged, indicating their limited ability to recognize emotional undertones.

### 6) What are the most striking errors made by the models?

Although sentences often ended in the pattern “... {mask}.” (with a period after the mask), models frequently predicted this period within the text as well. Multilingual BERT had particular difficulty with context, likely due to its training across many languages – it is unable to retain grammatical structures for each one, so its predictions often seem random.
The models frequently generated grammatically correct, but contextually incorrect, words. For example, in the sentence “Wczoraj w zoo przyglądałem się ogromnemu {mask}.” one would expect some animal rather than "zamieszaniu" or "zoo."
